In [1]:
!pip install -q -U kaggle_environments

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

# Агенты

## 1. Агент, играющий только "камень"

In [3]:
%%writefile rock_agent.py

def rock_only(observation, configuration):
    return 0

Writing rock_agent.py


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2. Агент, играющий только "бумагу"

In [4]:
%%writefile paper_agent.py

def paper_only(observation, configuration):
    return 1

Writing paper_agent.py


## 3. Агент, играющий только "ножницы"

In [5]:
%%writefile scissors_agent.py

def scissors_only(observation, configuration):
    return 2

Writing scissors_agent.py


## 4. Агент, играющий всегда случайный выбор

In [6]:
%%writefile random_agent.py
import random

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)

Writing random_agent.py


## 5. Агент, играющий выбор соперника в последнем раунде

In [7]:
%%writefile copy_agent.py
import random

def copier(observation, configuration):
    # делаем выбор, бьющий выбор соперника на предыдущем раунде
    if observation.step > 0:
        return observation.lastOpponentAction
    # если раунд первый, делаем случайный выбор
    else:
        return random.randrange(0, configuration.signs)

Writing copy_agent.py


## 6. Агент, меняющий вариант на выигрышный по выбору соперника в последнем раунде, если был проигрыш или ничья. <br>То есть если исход 0-0, то следующий будет 1-?.

In [8]:
%%writefile reactionary.py

import random

def count_score(agent, opponent):
    # функция, которая оценивает исход раунда и возвращает, 1-при выигрыше, 0-при ничье, (-1)-при проигрыше
    if not (agent + opponent) % 2:
        sub = opponent - agent # исходы 00, 11, 22, 02, 20
    else:
        sub = agent - opponent # исходы 01, 10, 12, 21
    if not sub:
        return 0
    elif sub > 0:
        return 1
    else:
        return -1

# переменная для хранения последнего выбора агента
last_react_action = None


def reactionary(observation, configuration):
    global last_react_action
    # если первый раунд, то случайны выбор
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    # если проигрыш или ничья, то меняем выбор на бьющий исход соперника
    elif count_score(last_react_action, observation.lastOpponentAction) < 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Writing reactionary.py


## 7. Агент, меняющий свой выбор на следующий после предположительного следующего по порядку выбора соперника, если в текущем раунде был выигрыш. <br>То есть если исход 0-0, то следующий будет 1-?, а если исход 1-0, то будет 0-?. На случай, если соперник захочет сыграть бьющий в предыдущем раунде выбор.

In [9]:
%%writefile future_reactionary.py

import random

def count_score(agent, opponent):
    # функция, которая оценивает исход раунда и возвращает, 1-при выигрыше, 0-при ничьи, (-1)-при проигрыше
    if not (agent + opponent) % 2:
        sub = opponent - agent # исходы 00, 11, 22, 02, 20
    else:
        sub = agent - opponent # исходы 01, 10, 12, 21
    if not sub:
        return 0
    elif sub > 0:
        return 1
    else:
        return -1

# переменная для хранения последнего выбора агента
last_counter_action = None


def future_reactionary(observation, configuration):
    global last_counter_action
    # если первый раунд, то случайны выбор
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    # если выигрыш, то меняем выбор на 2 вперед, на случай, если соперник выберет следующий исход (пример 1. 0-0; 2. 2-1)
    elif count_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = (last_counter_action + 2) % configuration.signs
    # в остальных случаях просто меняем выбор на следующий бьющий исход
    elif count_score(last_counter_action, observation.lastOpponentAction):
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Writing future_reactionary.py


## 8. Агент, который делает выбор за счет статистичеких данных, полученных из предыдущих раундов.

In [10]:
%%writefile statistical.py

import random

# переменная, где храним стат данные по исходам соперника
action_stats = {}

def statistical(observation, configuration):
    global action_stats
    # если первый раунд, то случайны выбор
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    action = observation.lastOpponentAction
    # добавляем выбор соперника в общую статистику
    if action not in action_stats:
        action_stats[action] = 0
    action_stats[action] += 1
    mode_action = None
    mode_action_count = None
    # Проходимся по статистике
    for k, v in action_stats.items():
        # Если встречаем число исходов соперника большее чем было на предыдущей итерации цикла, то фиксируем это
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v

    # возвращаем победный вариант, на самымй популярный выбор соперника
    return (mode_action + 1) % configuration.signs

Writing statistical.py


## 9. Агент, предсказывающий возможный выбор соперника за счет концепции марковских цепей. По последовательности исходов фиксированной длины подсчитывается наиболее вероятный исход соперника в следующем раунде.

In [11]:
%%writefile markov_agent.py

import numpy as np
import collections

# Переменные фиксированной последовательности и общей таблицы со статистиками выборов соперника при определенной последовательности
action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])

def markov_agent(observation, configuration):
    global table, action_seq

    # сформируем в первых раундах изначальную последовательность
    if len(action_seq) <= 5:
        action = np.random.randint(3)
        if observation.step > 0:
            action_seq.extend([observation.lastOpponentAction, action])
        else:
            action_seq.append(action)
        return action
    # создаем ключ к таблице из последовательности последних исходов
    key = ''.join([str(a) for a in action_seq[:-1]])
    # обновляем статистику в таблице в зависимости от исхода соперника (пример: на последовательности 1020101 соперник сыграл камень, значит меняем в таблице (1, 1, 1) на (2, 1, 1))
    table[key][observation.lastOpponentAction] += 1
    # обновляем нашу последовательность, сдвигая ее на один исход в раунде (два выбора)
    action_seq[:-2] = action_seq[2:]
    action_seq[-2] = observation.lastOpponentAction
    # формируем ключ для взятия статистики из таблицы
    key = ''.join([str(a) for a in action_seq[:-1]])
    scores = np.array(table[key])
    # формируем вероятность выбора соперника на основе дискретного распределения статистики
    next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum()) # в np.random.choice прописываем вероятности
    # создаем наш выбор, бьющий предполагаемый выбор соперника
    action = (next_opponent_action_pred + 1) % configuration.signs
    action_seq[-1] = action
    return int(action)

Writing markov_agent.py


# Прогон и результаты

In [47]:
# Формируем список агентов
list_names = [
    "rock_agent.py",
    "paper_agent.py",
    "scissors_agent.py",
    "random_agent.py",
    "copy_agent.py",
    "reactionary.py",
    "future_reactionary.py",
    "statistical.py",
    "markov_agent.py",
]

# создаем шаблон результаты игр для каждого агента
output = {}
for agent in list_names:
    output[agent] = {"win": 0, "loss": 0, "tie": 0}

In [48]:
# Делаем список игр между агентами
list_of_games = []
for i in list_names:
    for j in list_names:
        if i != j:
            list_of_games.append([i,j])

In [49]:
def get_result(agents):
    # функция, моделирующая игру и возвращающая названия агентов и результаты игры. Будет 1000 раундов в каждой игре
    output = evaluate(
                "rps",
                [agents[0], agents[1]],
                configuration={"episodeSteps": 1000})
    return agents[0], agents[1], output[0]

In [50]:
import tqdm
results = []
# проводим игры и записываем результаты
for game in tqdm.tqdm(list_of_games, total = len(list_of_games), desc="Processing"):
    content = get_result(game)
    results.append(content)

# обработаем результаты игры
for result in results:
    if result[2][0] == result[2][1] == 0.0:
        output[result[0]]['tie'] += 1
        output[result[1]]['tie'] += 1
    elif result[2][0] > result[2][1]:
        output[result[0]]['win']  += 1
        output[result[1]]['loss'] += 1
    else:
        output[result[0]]['loss'] += 1
        output[result[1]]['win']  += 1

Processing: 100%|██████████| 72/72 [02:59<00:00,  2.49s/it]


In [51]:
# формируем таблицу с результатами и сортируем по кол-ву выигрышей
pd.DataFrame.from_dict(output).T[['win','loss', 'tie']].sort_values('win', ascending=False)

,win,loss,tie
markov_agent.py,15,0,1
reactionary.py,9,5,2
statistical.py,8,4,4
random_agent.py,4,3,9
rock_agent.py,3,9,4
future_reactionary.py,3,3,10
paper_agent.py,2,10,4
scissors_agent.py,2,9,5
copy_agent.py,2,5,9


**Вывод:** из приведенной сверху таблицы, можно сделать вывод, что больше всего побед получил агент, работающий по концепции марковских цепей. При этом хочется отметить, что данный агент не разу не проиграл. Хуже всего себя повели константные(придерживающиеся одного выбора) агенты. Больше всего "ничья" получили агенты копирующий, рандомный и future_reactionary.